# Deep Q-Learning for Atari Breakout

## Reinforcement Learning

<img src='../img/RL01.png' width='600'>

- policy : $\pi(a|s)=P(a|s), \forall s, \forall a$

- value function : $v_{\pi}(s)=\sum P(z)R(z)=\sum_{a\in\mathbb{A}(s)}P(a|s)(r+v_{\pi}(s')), \forall s \in \mathbb{S}$

- reward : $R(z)=r_{t+1}+\gamma r_{t+2} + \gamma^2r_{t+3}+\cdots=\sum_{k=1}^{\infty}\gamma^{k-1}r_{t+k}$

- Q-Value : $Q_{\pi}(s,a)=E_{\pi}[R_{t+1}+\gamma R_{t+2}+\gamma^2R_{t+3}+\cdots|S_t=s,A_t=a]$

### Deep Q-Learning

Agent가 action을 수행하고 envrionment을 이동하면서 관찰된 상태를 action에 매핑.

Agent는 예상되는 가장 높은 장기 reward을 기반으로 가중치가 부여된 reward 인 "Q- 값"을 기반으로 주어진 상태에서 action을 선택.

Q-Learning Agent는 recommended action이 잠재적인 미래 reward를 극대화하도록 action을 수행하는 방법을 학습. 

이 방법은 "Off-Policy"방법으로 간주. 

즉, 최상의 작업이 선택되지 않은 경우에도 최상의 작업이 선택되었다고 가정하여 해당 Q 값이 업데이트

<img src='../img/dqn02.png' width='600'>

### Epsilon-Greedy Algorithm

- Greedy Algorithm은 미래를 생각하지 않고 각 단계에서 가장 최선의 선택을 하는 기법으로 각 단계에서 최선의 선택을 하는 것이 전체적으로도 최선이길 바라는 알고리즘

- Epsilon-Greedy Algorithm은 이를 개선시킨 전략, 일정한 확률로 랜덤으로 Action을 취하는 것. 

- Epsilon이라는 HyperParameter는 0~1 사이의 변수

- 만약 0에서 1 사이의 임의의 숫자를 얻었을 때, 𝜖 값보다 작다면, 임의의 행동을 선택

- 그리고 𝜖 값보다 크다면, 뉴럴 네트워크의 출력값에 근거해서 행동을 선택 

- 이 𝜖 값은 학습 초반에 크고 학습이 이루어 질수록 0에 가까운 값으로 작아진다. 

- 이렇게 해서, 학습의 초반에 더 많은 가능성들을 탐색하고, 학습의 후반에 뉴럴 네트워크가 좋은 정답을 출력할 수 있도록 한다.

### Atari Breakout

보드는 화면 하단을 따라 이동하여 화면 상단의 블록을 파괴 할 공을 반환. 

게임의 목적은 레벨의 모든 블록과 브레이크 아웃을 제거하는 것입니다. 

Agent는 공이 보드를 통과하지 않고 좌우로 움직이고 공을 되돌리고 모든 블록을 제거하여 보드를 제어하는 방법을 학습해야한다.

<img src='../img/breakout.jpeg' width='200'>

### Note

Deepmind 문서는 "총 5 천만 프레임 (즉, 총 38 일의 게임 경험)"에 대해 훈련. 

그러나 이 스크립트는 최신 컴퓨터에서 24 시간 이내에 처리되는 약 1 천만 프레임에서 좋은 결과를 제공.

## Setup

In [ ]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y
!pip install pyvirtualdisplay
!pip install piglet
!apt-get install git -y
!git clone https://github.com/openai/baselines.git
!pip install atari_py

In [1]:
import sys
sys.path.append('/tf/dsba/keras2torch/Reinforcement_Learning/KHW/baselines')
from baselines.common.atari_wrappers import make_atari, wrap_deepmind
import numpy as np
import pandas as pd
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 게임 이미지를 그리는 가상 디스플레이 생성
# Colab이나 Jupyter 같은 환경에서만 필요. 로컬은 필요 없음
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

bash: ../xvfb: No such file or directory
env: DISPLAY=:1


In [4]:
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # 감가율
alpha = 0.00025
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
max_steps_per_episode = 10000

# Use the Baseline Atari environment because of Deepmind helper functions
env = make_atari("BreakoutNoFrameskip-v4")
# Warp the frames, grey scale, stake four frame and scale to smaller ratio
env = wrap_deepmind(env, frame_stack=True, scale=True)
env.seed(seed)

[42, 742738649]

## Deep Q-Learning 네트워크 구현

<img src='../img/dqn01.png' width='600'>

In [5]:
num_actions = 4 # action
input_size = (84, 84, 4,) 
num_hidden = 128  # hidden layer node 수

class DQN(nn.Module):
    def __init__(self, input_size, num_actions, num_hidden, gamma=0.99):
        super(DQN, self).__init__()
        
        self.num_actions = num_actions
        self.dqn_layer = nn.Sequential(
            nn.Conv2d(in_channels = 4,
                      out_channels = 32,
                      kernel_size = 8,
                      stride = 4),
            nn.ReLU(),
            nn.Conv2d(in_channels = 32,
                      out_channels = 64,
                      kernel_size = 4,
                      stride = 2),
            nn.ReLU(),
            nn.Conv2d(in_channels = 64,
                     out_channels = 64,
                     kernel_size = 3,
                     stride = 1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(7*7*64,512),
            nn.Linear(512,self.num_actions)
        )

    def forward(self, input):
        return self.dqn_layer(input)

<img src='../img/dqn03.png' width='800'>

In [6]:
model = DQN(input_size, num_actions, num_hidden, gamma).to(device)
model_target = DQN(input_size, num_actions, num_hidden, gamma).to(device)
model_target.load_state_dict(model.state_dict())
model_target.eval()

optimizer = optim.Adam(model.parameters(), lr=alpha)

- Q function

$$Q^\pi(s,a)=E_{\pi}\{R_t|s_t=s, a_t=a\}=E_{\pi}\{\sum^T_{t'=t}\gamma^{t'-t}r_{t'}|s_t=s, a_t=a\}$$

- Optimal Q function

$$Q^*(s,a)=\underset{\pi}{\mathrm{max}}E\{R_t|s_t=s,a_t=a\}$$

- Optimal Q function (using Bellman equation)

$$Q^*(s,a)=E_{s'~\epsilon}[r+\gamma\underset{a'}{\mathrm{max}}Q^*(s',a')|s,a]$$

이 DQN의 가장 큰 contribution은 두 가지 아이디어로 Q-learning 알고리즘을 개선해서 neural network predictor 적용에 성공한 것. 

1. experience replay
    - 인접한 학습 데이터 사이의 correlation으로 인한 비효율성을 극복하기 위한 기법 
    - 게임을 하는 agent의 경험 데이터$(s,a,r,s')$를 replay memory라는 이름의 buffer pool에 매 순간 저장
    - update 할 때는 replay memory에서 random하게 minibatch 크기의 sample을 뽑아 계산하는 것


2. target network
    - DQN과 똑같은 neural network을 하나 더 만들어, 그 weight 값이 가끔씩만 update 되도록 한 것
    - $Q(s,a)$를 학습하는 순간, target 값도 따라 변하면서 학습 성능이 떨어지는 문제를 개선하기 위해서
    - Target network의 weight 값들은 주기적으로 DQN의 값을 복사. 
    - Q-learning의 update에서 아래 식과 같은 loss function을 사용하는데, 먼저 나오는 Q는 target network에서 계산한 것이고 뒤의 Q는 원래의 DQN에서 계산한 것
    
$$L_i(\theta_i)=E_{(s,a,r,s')~U(D)}[(r+\gamma\underset{a'}{\mathrm{max}}Q(s',a';\theta^{-}_i)-Q(s,a;\theta_i))^2]$$

Huber loss

일정한 범위(
δ)를 정해서 그 안에 있으면 오차를 제곱하고, 그 밖에 있으면 오차의 절대값을 구하는 것

$$L_{\delta}(e)=\left\{\begin{matrix}
 \frac{1}{2}e^2&\textrm{for}\left |  e\right | \leq \delta\\ 
\delta(\left | e \right |- \frac{1}{2}\delta), & \mathrm{otherwise}
\end{matrix}\right.$$

In [33]:
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0

# Number of frames to take random action and observe output
epsilon_random_frames = 50000
# Number of frames for exploration
epsilon_greedy_frames = 1000000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
max_memory_length = 100000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
update_target_network = 10000

while True:
    state = torch.FloatTensor(env.reset())
    state = torch.unsqueeze(state, 0).permute(0,3,1,2)
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1
        
        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            state = state.to(device)
            action_probs = model(state)
            action = action_probs.data.max(1)[1]
        
        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        state_next = torch.FloatTensor(state_next)
        state_next = torch.unsqueeze(state_next, 0).permute(0,3,1,2)
        
        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state.detach().cpu())
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next
        
        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = torch.cat([state_history[i] for i in indices])
            state_next_sample = torch.cat([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = torch.FloatTensor(
                [float(done_history[i]) for i in indices]
            )
            
            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            state_next_sample = torch.FloatTensor(state_next_sample).to(device)
            future_rewards = model_target(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            values, _ = torch.max(future_rewards, 1)
            updated_q_values = rewards_sample + gamma * values.detach().cpu().numpy()

            # If final frame set the last value to -1
            updated_q_values = torch.tensor(updated_q_values) * (1 - done_sample) - done_sample
            updated_q_values = updated_q_values.to(device)

            # Create a mask so we only calculate loss on the updated Q-values
            masks = nn.functional.one_hot(torch.tensor(action_sample).to(device), num_actions)

            optimizer.zero_grad()
            
            # Train the model on the states and updated Q-values
            state_sample = torch.FloatTensor(state_sample).to(device)
            q_values = model(state_sample)
            # Apply the masks to the Q-values to get the Q-value for action taken
            q_action = torch.sum(torch.multiply(q_values, masks), 1, dtype=torch.float64)
            loss = F.smooth_l1_loss(updated_q_values, q_action)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()


        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            model_target.load_state_dict(model.state_dict())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break
    
    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 40:  # Condition to consider the task solved
        print("Solved at episode {}!".format(episode_count))
        break

running reward: 0.30 at episode 292, frame count 10000
running reward: 0.30 at episode 568, frame count 20000
running reward: 0.19 at episode 892, frame count 30000
running reward: 0.36 at episode 1178, frame count 40000
running reward: 0.32 at episode 1457, frame count 50000
running reward: 0.50 at episode 1694, frame count 60000
running reward: 0.66 at episode 1917, frame count 70000
running reward: 0.51 at episode 2150, frame count 80000
running reward: 0.52 at episode 2385, frame count 90000
running reward: 0.62 at episode 2591, frame count 100000
running reward: 0.59 at episode 2809, frame count 110000
running reward: 0.66 at episode 3011, frame count 120000
running reward: 0.62 at episode 3222, frame count 130000
running reward: 0.76 at episode 3410, frame count 140000
running reward: 0.71 at episode 3599, frame count 150000
running reward: 0.73 at episode 3794, frame count 160000
running reward: 0.81 at episode 3965, frame count 170000
running reward: 0.68 at episode 4156, frame

running reward: 6.48 at episode 14118, frame count 1430000
running reward: 6.73 at episode 14156, frame count 1440000
running reward: 6.08 at episode 14206, frame count 1450000
running reward: 5.98 at episode 14250, frame count 1460000
running reward: 5.78 at episode 14300, frame count 1470000
running reward: 5.94 at episode 14343, frame count 1480000
running reward: 6.49 at episode 14383, frame count 1490000
running reward: 6.83 at episode 14424, frame count 1500000
running reward: 6.32 at episode 14473, frame count 1510000
running reward: 6.01 at episode 14516, frame count 1520000
running reward: 6.23 at episode 14559, frame count 1530000
running reward: 6.76 at episode 14602, frame count 1540000
running reward: 6.02 at episode 14648, frame count 1550000
running reward: 6.38 at episode 14687, frame count 1560000
running reward: 6.59 at episode 14729, frame count 1570000
running reward: 6.46 at episode 14772, frame count 1580000
running reward: 6.80 at episode 14811, frame count 15900

running reward: 6.96 at episode 20361, frame count 2820000
running reward: 7.52 at episode 20400, frame count 2830000
running reward: 7.41 at episode 20441, frame count 2840000
running reward: 7.24 at episode 20480, frame count 2850000
running reward: 7.25 at episode 20520, frame count 2860000
running reward: 7.45 at episode 20561, frame count 2870000
running reward: 8.04 at episode 20598, frame count 2880000
running reward: 6.70 at episode 20650, frame count 2890000
running reward: 5.89 at episode 20697, frame count 2900000
running reward: 6.55 at episode 20737, frame count 2910000
running reward: 6.94 at episode 20779, frame count 2920000
running reward: 5.23 at episode 20838, frame count 2930000
running reward: 5.23 at episode 20883, frame count 2940000
running reward: 6.31 at episode 20925, frame count 2950000
running reward: 6.47 at episode 20966, frame count 2960000
running reward: 7.05 at episode 21007, frame count 2970000
running reward: 6.80 at episode 21053, frame count 29800

KeyboardInterrupt: 

----

## Visualization

In [34]:
# Render an episode and save as a GIF file

from IPython import display as ipythondisplay
from PIL import Image
from pyvirtualdisplay import Display


display = Display(visible=0, size=(400, 300))
display.start()


def render_episode(env, model, max_steps): 
    screen = env.render(mode='rgb_array')
    im = Image.fromarray(screen)

    images = [im]
  
    state = env.reset()
    for i in range(1, max_steps + 1):
        state = torch.FloatTensor(state).to(device)
        state = torch.unsqueeze(state, 0).permute(0,3,1,2)
        action_probs = model(state)
        action = action_probs.data.max(1)[1]

        state, _, done, _ = env.step(action.item())

        # Render screen every 10 steps
        if i % 10 == 0:
            screen = env.render(mode='rgb_array')
            images.append(Image.fromarray(screen))
  
        if done:
            break
  
    return images


# Save GIF image
images = render_episode(env, model, max_steps_per_episode)
image_file = 'BreakoutNoFrameskip-v4.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)

- Before any training: 

<img src='../img/breakout01.gif' width='300'>

- n early stages of training: 

<img src='../img/breakout02.gif' width='300'>

- In later stages of training:

<img src='../img/breakout03.gif' width='300'>

------